In [68]:
!pip3 install --upgrade --quiet langchain langchain-openai langchain-community chromadb
!pip3 install --upgrade --quiet streamlit pandas python-dotenv pypdf
!pip3 install --upgrade --quiet  faiss-cpu

In [ ]:
#Importação de documento pdf, vetorização, conexão com a OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma, FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
import uuid

#enviroment
from dotenv import load_dotenv

In [70]:
load_dotenv()

True

In [71]:
loader = PyPDFLoader("pdf/codigo_penal.pdf")
pages = loader.load()
pages

[Document(metadata={'producer': 'Adobe PDF Library 16.0.7', 'creator': 'Adobe InDesign 17.4 (Windows)', 'creationdate': '2023-03-30T16:36:24-03:00', 'moddate': '2023-03-30T16:36:24-03:00', 'title': '', 'source': 'pdf/codigo_penal.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}, page_content='Código Penal\n6a edição\nAtualizada até fevereiro de 2023'),
 Document(metadata={'producer': 'Adobe PDF Library 16.0.7', 'creator': 'Adobe InDesign 17.4 (Windows)', 'creationdate': '2023-03-30T16:36:24-03:00', 'moddate': '2023-03-30T16:36:24-03:00', 'title': '', 'source': 'pdf/codigo_penal.pdf', 'total_pages': 153, 'page': 1, 'page_label': '2'}, page_content='SENADO FEDERAL\nMesa\nBiênio 2023 – 2024\nSenador Rodrigo Pacheco\nPRESIDENTE\nSenador Veneziano Vital do Rêgo\nPRIMEIRO-VICE-PRESIDENTE\nSenador Rodrigo Cunha\nSEGUNDO-VICE-PRESIDENTE\nSenador Rogério Carvalho\nPRIMEIRO-SECRETÁRIO\nSenador Weverton\nSEGUNDO-SECRETÁRIO\nSenador Chico Rodrigues\nTERCEIRO-SECRETÁRIO\nSenador Styvenson Va

In [72]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200,
                                               length_function=len,
                                               separators=["\n\n", "\n", " "])

chunks = text_splitter.split_documents(pages)

In [73]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002"
    )
    return embeddings

embedding_function = get_embedding_function()

In [75]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [76]:
def create_vectorstore(chunks, embedding_function, vectorstore_path):
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    unique_ids = set()
    unique_chunks = []

    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)

    vectorstore = Chroma.from_documents(documents=unique_chunks,
                                        embedding=embedding_function,
                                        persist_directory=vectorstore_path)
    return vectorstore

In [77]:
vectorstore = create_vectorstore(chunks=chunks,
                                 embedding_function=embedding_function,
                                 vectorstore_path="vectorstore")

In [78]:
vectorstore = Chroma(persist_directory="vectorstore", embedding_function=embedding_function)

In [95]:
PROMPT_TEMPLATE="""
Você é um assistente feito para criar perguntas e respostas.
Crie uma base com de 35 a 40 questões e respostas.
Coloque no final de cada resposta o símbolo ||
Use os seguintes pedaços do texto que foram recuperados para criar as perguntas e as respostas.

{context}

---

Faça utilizando o seguinte formato:
Pergunta: ____
Resposta: ____
"""

In [96]:
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=chunks)
print(prompt)

Human: 
Você é um assistente feito para criar perguntas e respostas.
Crie uma base com de 35 a 40 questões e respostas.
Coloque no final de cada resposta o símbolo ||
Use os seguintes pedaços do texto que foram recuperados para criar as perguntas e as respostas.

[Document(metadata={'producer': 'Adobe PDF Library 16.0.7', 'creator': 'Adobe InDesign 17.4 (Windows)', 'creationdate': '2023-03-30T16:36:24-03:00', 'moddate': '2023-03-30T16:36:24-03:00', 'title': '', 'source': 'pdf/codigo_penal.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}, page_content='Código Penal\n6a edição\nAtualizada até fevereiro de 2023'), Document(metadata={'producer': 'Adobe PDF Library 16.0.7', 'creator': 'Adobe InDesign 17.4 (Windows)', 'creationdate': '2023-03-30T16:36:24-03:00', 'moddate': '2023-03-30T16:36:24-03:00', 'title': '', 'source': 'pdf/codigo_penal.pdf', 'total_pages': 153, 'page': 1, 'page_label': '2'}, page_content='SENADO FEDERAL\nMesa\nBiênio 2023 – 2024\nSenador Rodrigo Pacheco\nPRESIDE

In [ ]:
generated_qa = llm.invoke(prompt)

print(generated_qa)

content='Claro! Aqui está uma lista de perguntas e respostas com base no texto fornecido, seguindo o formato que você especificou:\n\n1. Pergunta: Qual é a edição do Código Penal mencionado?\n   Resposta: A 6ª edição, atualizada até fevereiro de 2023. ||\n\n2. Pergunta: Quem é o presidente do Senado Federal para o biênio 2023-2024?\n   Resposta: Senador Rodrigo Pacheco. ||\n\n3. Pergunta: Qual é o número do Decreto-lei que institui o Código Penal no Brasil?\n   Resposta: Decreto-lei nº 2.848 de 1940. ||\n\n4. Pergunta: O que aborda a Parte Geral do Código Penal?\n   Resposta: A Parte Geral do Código Penal aborda a aplicação da lei penal, do crime, da imputabilidade penal e das penas. ||\n\n5. Pergunta: Qual é a pena para o crime de homicídio simples, conforme o Código Penal?\n   Resposta: A pena é de reclusão, de seis a vinte anos. ||\n\n6. Pergunta: O que caracteriza o homicídio qualificado?\n   Resposta: O homicídio qualificado é caracterizado por motivos torpes, emprego de veneno, t

In [122]:
import re

split_qa = generated_qa.content.split("||\n")

split_qa


['Claro! Aqui está uma lista de perguntas e respostas com base no texto fornecido, seguindo o formato que você especificou:\n\n1. Pergunta: Qual é a edição do Código Penal mencionado?\n   Resposta: A 6ª edição, atualizada até fevereiro de 2023. ',
 '\n2. Pergunta: Quem é o presidente do Senado Federal para o biênio 2023-2024?\n   Resposta: Senador Rodrigo Pacheco. ',
 '\n3. Pergunta: Qual é o número do Decreto-lei que institui o Código Penal no Brasil?\n   Resposta: Decreto-lei nº 2.848 de 1940. ',
 '\n4. Pergunta: O que aborda a Parte Geral do Código Penal?\n   Resposta: A Parte Geral do Código Penal aborda a aplicação da lei penal, do crime, da imputabilidade penal e das penas. ',
 '\n5. Pergunta: Qual é a pena para o crime de homicídio simples, conforme o Código Penal?\n   Resposta: A pena é de reclusão, de seis a vinte anos. ',
 '\n6. Pergunta: O que caracteriza o homicídio qualificado?\n   Resposta: O homicídio qualificado é caracterizado por motivos torpes, emprego de veneno, tra

In [153]:
qa_final = []

for i in split_qa:
    qa_question = re.findall(r"Pergunta: \s*(.*?\?)", i)
    qa_answer = re.findall(r"Resposta: \s*(.*.\.)", i)
    
    qa_question = ' '.join(qa_question)
    qa_answer = ' '.join(qa_answer)

    if qa_question and qa_answer:
        qa_final.append({"question": qa_question, "answer": qa_answer})

qa_final

[{'question': 'Qual é a edição do Código Penal mencionado?',
  'answer': 'A 6ª edição, atualizada até fevereiro de 2023.'},
 {'question': 'Quem é o presidente do Senado Federal para o biênio 2023-2024?',
  'answer': 'Senador Rodrigo Pacheco.'},
 {'question': 'Qual é o número do Decreto-lei que institui o Código Penal no Brasil?',
  'answer': 'Decreto-lei nº 2.848 de 1940.'},
 {'question': 'O que aborda a Parte Geral do Código Penal?',
  'answer': 'A Parte Geral do Código Penal aborda a aplicação da lei penal, do crime, da imputabilidade penal e das penas.'},
 {'question': 'Qual é a pena para o crime de homicídio simples, conforme o Código Penal?',
  'answer': 'A pena é de reclusão, de seis a vinte anos.'},
 {'question': 'O que caracteriza o homicídio qualificado?',
  'answer': 'O homicídio qualificado é caracterizado por motivos torpes, emprego de veneno, traição, entre outros.'},
 {'question': 'O que é considerado furto, segundo o artigo 155 do Código Penal?',
  'answer': 'Subtrair co

In [155]:
import pandas as pd

qa_df = pd.DataFrame(qa_final)

qa_df

,question,answer
0,Qual é a edição do Código Penal mencionado?,"A 6ª edição, atualizada até fevereiro de 2023."
1,Quem é o presidente do Senado Federal para o b...,Senador Rodrigo Pacheco.
2,Qual é o número do Decreto-lei que institui o ...,Decreto-lei nº 2.848 de 1940.
3,O que aborda a Parte Geral do Código Penal?,A Parte Geral do Código Penal aborda a aplicaç...
4,Qual é a pena para o crime de homicídio simple...,"A pena é de reclusão, de seis a vinte anos."
5,O que caracteriza o homicídio qualificado?,O homicídio qualificado é caracterizado por mo...
6,"O que é considerado furto, segundo o artigo 15...","Subtrair coisa alheia móvel, para si ou para o..."
7,Qual é a pena para o crime de roubo?,"A pena é de reclusão, de quatro a dez anos, e ..."
8,"O que constitui a extorsão, de acordo com o Có...","Constranger alguém, mediante violência ou grav..."
9,O que é considerado como concurso de pessoas n...,"Quem, de qualquer modo, concorre para o crime ..."


In [158]:
qa_df.to_csv('dataframe/df_codigo_penal.csv')